# Markov Chains
## Introduction
https://en.wikipedia.org/wiki/Markov_chain  
From Wikipedia: A Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event.  

Mathematically a Markov chain can be described as follows:  
Given a set of discrete states $s_i \in S$ and events defined as discrete times with an assigned state $X_t=(t,s_i)$, $t \in \mathbb{Z}$  
A Markov chain defines a model for a sequence of events as a probability distribution based on a previous event's state:  
$$ \Pr \left (X_{n+1}=(n+1,s_i) \mid X_{n}=(n,s_j) \right ) = p_{i,j} $$  
Such that
$$ \sum_{j} p_{i,j} = 1 $$  
Note that $p_{i,j}$ defines a stochastic state transition matrix  
https://en.wikipedia.org/wiki/Stochastic_matrix


## Basic Example

In [ ]:
# Imports
# Data reading and wrangling
import pandas as pd
import numpy as np
import re

# The text Markov parser and generator
import markovify

# Graph library
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph

# Drawing and rendering
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)

# Image display
from IPython.display import Image

In [ ]:
# Define a simple Markov chain
edge_list = [
    ('A', 'B', {'weight':0.2}),
    ('A', 'C', {'weight':0.4}),
    ('A', 'D', {'weight':0.4}),
    ('B', 'A', {'weight':0.5}),
    ('B', 'C', {'weight':0.5}),
    ('C', 'B', {'weight':0.5}),
    ('C', 'C', {'weight':0.5}),
    ('D', 'C', {'weight':1.0})
]
G_example = nx.MultiDiGraph(edge_list)

In [ ]:
G_example_PGV = to_agraph(G_example)
G_example_PGV.node_attr['shape'] = 'circle'
G_example_PGV.graph_attr['splines'] = 'true'
G_example_PGV.graph_attr['esep'] = '2'
for e1,e2,edge_data in G_example.edges(data=True):
    G_example_PGV.get_edge(e1, e2).attr['label'] = edge_data['weight']
    
G_example_PGV.layout('circo')
Image(G_example_PGV.draw(format='png'))

In [ ]:
print(list(G_example.nodes()))
nx.to_numpy_matrix(G_example)

## Autocomplete and Text Generation
Using Markov chains we can essencially create a probablity distribution over possible sentences created by chaining words that come after one-another, and then sample from this distribution to generate text!  
### Text Generation Basic Example

In [ ]:
text_model = markovify.Text("I am One. I am Two. I am Three.", state_size=1)
text_model.chain.model

In [ ]:
text_model.make_sentence(test_output=False)

In [ ]:
# Turn into a dict with probabilities instead of counts
G_dict = [(''.join(word),child_word,{'weight':float(weight)/sum(children_words.values())}) for word,children_words in text_model.chain.model.items() for child_word,weight in children_words.items()]
# Turn into a NetworkX object
G = nx.MultiDiGraph(G_dict)

In [ ]:
list(G.edges(data=True))

In [ ]:
G_example_PGV = to_agraph(G)
G_example_PGV.node_attr['shape'] = 'circle'
G_example_PGV.graph_attr['splines'] = 'true'
G_example_PGV.graph_attr['esep'] = '2'
for e1,e2,edge_data in G.edges(data=True):
    G_example_PGV.get_edge(e1, e2).attr['label'] = round(edge_data['weight'], 3)

G_example_PGV.layout('dot')
Image(G_example_PGV.draw(format='png'))

In [ ]:
print(list(G.nodes()))
nx.to_numpy_matrix(G)

### Text Generation Lyrics Example
**NOTE**: For this example you will need to obtain a dataset of lyics (newline separated, starting with the line 'lyrics'). Choose your favorite artist and grab a copy from your favorite lyrics source! A dataset has not been provided due to copyright concerns.

In [ ]:
df = pd.read_csv('lyrics.csv').fillna('')
lyrics_corpus = '\n'.join(df['lyrics'].tolist())
lyrics_corpus = lyrics_corpus.split('\n')

In [ ]:
lyrics_model = markovify.Text(lyrics_corpus, state_size=2)

In [ ]:
for i in range(10):
    print(lyrics_model.make_sentence())

### Autocomplete Lyrics Example
**NOTE**: See above  
The Markovian model is the very idea behind autocomplete on your phone: based upon the last few words you wrote, what are you most likely to write next?

In [ ]:
# This time we're only going to look for the next word after 1
lyrics_model = markovify.Text(lyrics_corpus, state_size=1)

In [ ]:
# List the top most frequently used following words
sorted(list(lyrics_model.chain.model[('___BEGIN__',)].items()), key=lambda x: x[1], reverse=True)

## PageRank
PageRank is the algorithm invented by Google founders Larry Page and Sergey Brin and is one of the primary metrics used by the Google search engine.  
The PageRank algorithm models a user's path through pages on the internet by clicking links as a Markov chain. The algorithm seeks to find the likelihood of a user landing on a given page. It is essentially a form of **eigencentrality** of the Markov chain's graph.  
https://en.wikipedia.org/wiki/PageRank  

### Mathematical Preliminary
Given a probability distribution over the states in a Markov chain at time $t$ as a row vector $\vec{s}_t$ the probability distribution over the states at time $t+1$ is given in by the stochastic state transition matrix $\bar{P}$:  
$$\vec{s}_{t+1} = \vec{s}_{t} \bar{P}$$

### Definition of Eigencentrality
A row eigenvector of the matrix $\bar{P}$ is defined by the equation:  
$$\vec{v} \bar{P} = \lambda\vec{v}$$  
If $\lambda = 1 $ then $\vec{v}$ is fixed point of the matrix multiplication with $\bar{P}$.  
What does this mean for a Markov chain? A fixed point would be a probability distribution of ending up in any of the states after infinite transitions: the long term probability.

### Definition of PageRank and the Google Matrix
PageRank models a person surfing the web by clicking on links between pages. For simplicity, they are assumed to have an equal chance of clicking on any link on a page. Sounds like a Markov chain! They are also assumed to only have a probability $\alpha$ of continuing to surf at any given step. Given a "universe" of web pages and the links between them, how do we find the probability of landing on a given page?  
We can accomplish this by adding a "quit transitioning" term to our eigenvector formula:  
$$\vec{R} = \alpha\vec{R}\bar{P} + \frac{1-\alpha}{N}\vec{1}$$
Where $\vec{1}$ is a vector of all $1$s  

So how do we compute the solution do this? Well since $\vec{R}$ is a probability distribution, its components must sum to 1 (i.e. the $L^{1}$ norm is $1$), therefore we can rewrite the PageRank expression as:  
$$\vec{R} = \vec{R} \left ( \alpha\bar{P} + \frac{1-\alpha}{N}\bar{1} \right ) = \vec{R} \bar{G}$$
Where $\bar{1}$ is a matrix of all $1$s  
$\bar{G}$ is what's known as the Google matrix.  

And so we're back to an eigenvector problem, which can be solved by usual eigenvector solving techniques. In the case of the PageRank problem, fixed point iteration converages quite rapidly.    
https://en.wikipedia.org/wiki/Google_matrix  
https://en.wikipedia.org/wiki/Fixed-point_iteration  
### A Demonstration

In [ ]:
# We're not going to write a spider to crawl the web for this demo
edge_list = [
    ('The Popular Kid', 'Follower1'),
    ('The Popular Kid', 'Best Friend'),
    ('The Popular Kid', 'Follower2'),
    ('Follower1', 'The Popular Kid'),
    ('Follower1', 'Best Friend'),
    ('Best Friend', 'The Popular Kid'),
    ('Follower2', 'The Popular Kid'),
    ('Follower2','Best Friend')
]
PR_graph = nx.MultiDiGraph(edge_list)

In [ ]:
G_example_PGV = to_agraph(PR_graph)
G_example_PGV.node_attr['shape'] = 'circle'
G_example_PGV.graph_attr['splines'] = 'true'
G_example_PGV.graph_attr['esep'] = '2'

G_example_PGV.layout('circo')
Image(G_example_PGV.draw(format='png'))

The NetworkX library actually comes with functions to perform the PageRank algorithm and compute the Google matrix

In [ ]:
print(list(PR_graph.nodes()))
print(nx.to_numpy_matrix(PR_graph))
nx.google_matrix(PR_graph, alpha=0.85)

In [ ]:
nx.pagerank_numpy(PR_graph, alpha=0.85)

But the PageRank algorithm is simple enough we can compute it manually as well

In [ ]:
alpha = 0.85
P = nx.to_numpy_matrix(PR_graph)
P = P/np.sum(P,axis=1)
Google_matrix = alpha * P + (1-alpha)/P.shape[0] * np.ones(P.shape)
Google_matrix

In [ ]:
iterations = 100
v = np.ones(P.shape[0])/P.shape[0]
for i in range(iterations):
    v = v * Google_matrix
v

## Conclusion
In this presentation we learned about Markov chains, starting with their mathematical definition, then we saw an example displayed as a graph.  
We then saw an example of a graph for text generation, and discussed its relevancy to autocomplete.  
Finally we looked at Google's classic PageRank algorithm in terms of Markov Chains for modeling a user surfing through a network.  
Hopefully this presentation has given useful insights and intuitions behind the ubiquitous concept of Markov Chains.